In [ ]:
df = spark.sql("SELECT * FROM bing_lake_db.bing_news_table WHERE date_time BETWEEN curdate()-1 AND curdate()")

In [ ]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

In [ ]:
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

In [ ]:
result = model.transform(df)

In [ ]:
# create sentiment column
from pyspark.sql.functions import col
sentiment_df=result.withColumn("sentiment",col("response.documents.sentiment"))

In [ ]:
sentiment_df=sentiment_df.drop("error","response")

In [ ]:
sentiment_df=sentiment_df.dropDuplicates()
display(sentiment_df)

In [ ]:
from pyspark.sql.utils import AnalysisException

try:
    table_name='bing_lake_db.bing_news_sentiment'
    sentiment_df.write.format("delta").saveAsTable(table_name)
except AnalysisException:
    print("Table Already Exists")
    sentiment_df.createOrReplaceTempView("vw_sentiment_df")
    spark.sql(f"""merge into {table_name} target_table
                using vw_sentiment_df source_view
                on source_view.title=target_table.title
                when matched and
                target_table.date_time<>source_view.date_time or
                target_table.description<>source_view.description or
                target_table.category<>source_view.category or
                target_table.provider<>source_view.provider
                then update set *
                when not matched then insert *
                """)